# I/O
> Functions to handle reading and writing datasets and model descriptions

In [ ]:
#| default_exp io

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| exporti
import json, os
import itertools as it
from collections import defaultdict

import numpy as np
import pandas as pd
import datetime as dt

from typing import List, Tuple, Dict, Union, Optional

import pyarrow as pa
import pyarrow.parquet as pq

import salk_toolkit as stk
from salk_toolkit.utils import replace_constants

In [ ]:
#| export

# Default usage with mature metafile: read_annotated_data(<metafile name>)
# When figuring out the metafile, it can also be run as: read_annotated_data(meta=<dict>, data_file=<>)
def read_annotated_data(meta_fname=None, multilevel=False, meta=None, data_file=None, return_meta=False):
    
    # Read metafile
    if meta_fname:
        with open(meta_fname,'r') as jf:
            meta = json.load(jf)
    
    # Setup constants with a simple replacement mechanic
    replace_constants(meta)
    
    # Read datafile
    if not data_file:
        data_file = os.path.join(os.path.dirname(meta_fname),meta['file'])
    opts = meta['read_opts'] if'read_opts' in meta else {}
    raw_data = pd.read_csv(data_file, **opts)
    
    res = []
    

    
    if 'preprocessing' in meta:
        exec(meta['preprocessing'],{'pd':pd, 'np':np },{ 'df':raw_data })
        
    for group in meta['structure']:
        gres = []
        for tpl in group['columns']:
            if type(tpl)==list:
                cn = tpl[0] # column name
                sn = tpl[1] if type(tpl[1])==str else cn # source column
                cd = tpl[2] if len(tpl)==3 else tpl[1] if type(tpl[1])==dict else {} # metadata
            else:
                cn = sn = tpl
                cd = {}

            if 'scale' in group: cd = {**group['scale'],**cd}
            if sn not in raw_data:
                print(f"Column {sn} not found")
                continue
            s = raw_data[sn].rename(cn)
            
            if 'translate' in cd: s.replace(cd['translate'],inplace=True)
            
            if 'transform' in cd: s = eval(cd['transform'],{ 's':s, 'df':raw_data, 'pd':pd, 'np':np })
            
            if 'categories' in cd: 
                na_sum = s.isna().sum()
                cats = cd['categories'] if cd['categories']!='infer' else [ c for c in s.unique() if pd.notna(c) ]
                s = pd.Series(pd.Categorical(s,categories=cats,ordered=cd['ordered'] if 'ordered' in cd else False), name=cn)
                # Check if the category list provided was comprehensive
                new_nas = s.isna().sum() - na_sum
                if new_nas > 0: print(f'Column {cn} has {new_nas} entries that were not listed in categories')
            gres.append(s)
        if len(gres)==0: continue
        gdf = pd.concat(gres,axis=1)
        gdf.columns = pd.MultiIndex.from_arrays([[group['name']]*len(gdf.columns),gdf.columns])
        res.append(gdf)
    
    df = pd.concat(res,axis=1)
    
    if 'postprocessing' in meta:
        exec(meta['postprocessing'],{'pd':pd, 'np':np },{ 'df': df })

    if not multilevel:
        df.columns = df.columns.get_level_values(1)    
    
    return (df, meta) if return_meta else df

In [ ]:
#| export
def read_and_process_data(desc, return_meta=False):
    data, meta = read_annotated_data(desc['file'],return_meta=True)
    
    # Perform transformation and filtering
    if 'preprocessing' in desc: exec(desc['preprocessing'],  {'pd':pd, 'np':np },{ 'df':data })
    if 'filter' in desc: data = data[eval(desc['filter'],    {'pd':pd, 'np':np },{ 'df':data })]
    if 'postprocessing' in desc: exec(desc['postprocessing'],{'pd':pd, 'np':np },{ 'df':data })
    
    return (data, meta) if return_meta else data

In [ ]:
dataf_meta = {
    'file': '../data/master_meta.json',
    'preprocessing': "df.age_group.replace({'16-24':'18-24'}, inplace=True)",
    'filter': '(df.citizen) & (df.age>=18) & (df.wave>df.wave.max()-5)',
}

df = read_and_process_data(dataf_meta)

FileNotFoundError: [Errno 2] No such file or directory: '../data/master_meta.json'

In [ ]:
#| export
def save_population_h5(fname,pdf):
    hdf = pd.HDFStore(fname,complevel=9, complib='zlib')
    hdf.put('population',pdf,format='table')
    hdf.close()
    
def load_population_h5(fname):
    hdf =  pd.HDFStore(fname, mode='r')
    res = hdf['population'].copy()
    hdf.close()
    return res

In [ ]:
#| export
def save_sample_h5(fname,trace,COORDS = None, filter_df = None):
    odims = [d for d in trace.predictions.dims if d not in ['chain','draw','obs_idx']]
    
    if COORDS is None: # Recover them from trace (requires posterior be saved in same trace)
        inds = trace.posterior.indexes
        coords = { t: list(inds[t]) for t in inds if t not in ['chain','draw'] and '_dim_' not in t}
        COORDS = { 'immutable': coords, 'mutable': ['obs_idx'] }

    if filter_df is None: # Recover filter dimensions and data from trace (works only for GLMs)
        rmdims = odims + list({'time','unit','combined_inputs'} & set(trace.predictions_constant_data.dims))
        df = trace.predictions_constant_data.drop_dims(rmdims).to_dataframe()#.set_index(demographics_order).indexb
        df.columns = [ s.removesuffix('_id') for s in df.columns]
        df.drop(columns=[c for c in df.columns if c[:4]=='obs_'],inplace=True)

        for d in df.columns:
            if d in COORDS['immutable']:
                fs = COORDS['immutable'][d]
                df[d] = pd.Categorical(df[d].replace(dict(enumerate(fs))),fs)
                if d in orders: df[d] = pd.Categorical(df[d],orders[d],ordered=True)
        filter_df = df

    chains, draws = trace.predictions.dims['chain'], trace.predictions.dims['draw']
    dinds = np.array(list(it.product( range(chains), range(draws), list(filter_df.index)))).reshape( (-1, 3) )

    res_dfs = { 'filter': filter_df }
    for odim in odims:
        response_cols = list(np.array(trace.predictions[odim]))
        xdf = pd.DataFrame(np.concatenate( (
            dinds,
            np.array(trace.predictions['y_'+odim]).reshape( ( -1,len(response_cols) ) )
            ), axis=-1), columns = ['chain', 'draw', 'obs_idx'] + response_cols)
        res_dfs[odim] = postprocess_rdf(xdf,odim)
        
    # Save dfs as hdf5
    hdf = pd.HDFStore(fname,complevel=9, complib='zlib')
    for k,vdf in res_dfs.items():
        hdf.put(k,vdf,format='table')
    hdf.close()


In [ ]:
#| export

# These two very helpful functions are borrowed from https://towardsdatascience.com/saving-metadata-with-dataframes-71f51f558d8e

custom_meta_key = 'salk-toolkit-meta'

def save_parquet_with_metadata(df, meta, file_name):
    table = pa.Table.from_pandas(df)
    
    custom_meta_json = json.dumps(meta)
    existing_meta = table.schema.metadata
    combined_meta = {
        custom_meta_key.encode() : custom_meta_json.encode(),
        **existing_meta
    }
    table = table.replace_schema_metadata(combined_meta)
    
    pq.write_table(table, file_name, compression='GZIP')
    
def load_parquet_with_metadata(file_name):
    restored_table = pq.read_table(file_name)
    restored_df = restored_table.to_pandas()
    restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
    restored_meta = json.loads(restored_meta_json)
    
    return restored_df, restored_meta

In [ ]:
# Test saving and loading parquet with metadata
df = pd.DataFrame([[1,2],[3,4]],columns=['x','y'])
meta = { 'test': [1,{'x':2},[3]] }

save_parquet_with_metadata(df,meta,'test.parquet')
ndf, nmeta = load_parquet_with_metadata('test.parquet')

assert nmeta == meta
assert ndf.equals(df)

In [ ]:
# This is how you can add tests
#assert False

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()